In [2]:
import gradio as gr
import torch
import torchtext

class MLPClassifier(torch.nn.Module):
    def __init__(self, word_vec, embedding_size):
        super().__init__()
        self.embed=torch.nn.Embedding.from_pretrained(word_vec)
        self.fc1 = torch.nn.Linear(embedding_size, 1)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        torch.manual_seed(26)
        if isinstance(module, torch.nn.Linear):
            module.weight.data.normal_(0, 0.3)
            #print(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()

    def forward(self, input):
        squash=torch.nn.Sigmoid()
        pred= self.embed(input)
        pred= torch.mean(pred, dim=0)
        pred= squash(self.fc1(pred))
        return pred

class CNNClassifier(torch.nn.Module):
    def __init__(self, word_vec,embedding_size,k1,n1,k2,n2):
        super().__init__()
        self.embed=torch.nn.Embedding.from_pretrained(word_vec,freeze=False)
        self.conv1=torch.nn.Conv2d(1, n1, (k1,embedding_size),bias=False)
        self.conv2=torch.nn.Conv2d(1, n2, (k2,embedding_size),bias=False)
        self.maxpool1=torch.nn.AdaptiveMaxPool1d(1)
        self.maxpool2=torch.nn.AdaptiveMaxPool1d(1)
        self.fc1 = torch.nn.Linear(n1+n2, 1)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        torch.manual_seed(26)
        if isinstance(module, torch.nn.Linear):
            module.weight.data.normal_(0, 0.3)
            #print(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()

    def forward(self, input):
        #B=batch size S=sentence length E=embedding size 
        #input=[S,B,E]
        squash=torch.nn.ReLU()
        binary=torch.nn.Sigmoid()
        pred= self.embed(input.swapaxes(0,1))#swap sentence length and batch size axis [B,S,E]
        pred=pred.unsqueeze(1)#[B,1,S,E]

        pred1= squash(self.conv1(pred))#[B,n1,S,1]
        pred1=torch.squeeze(pred1,3)#[B,n1,S]
        pred1=self.maxpool1(pred1)#[B,n1,1]

        pred2= squash(self.conv2(pred))#[B,n2,S,1]
        pred2=torch.squeeze(pred2,3)#[B,n2,S]
        pred2=self.maxpool2(pred2)#[B,n2,1]

        pred=torch.squeeze(torch.cat((pred1,pred2),1),2)#[B,n1+n2]
        pred= binary(self.fc1(pred))
        return pred

glove = torchtext.vocab.GloVe(name="6B", dim=100)

checkpoint = torch.load('baseline.pt')
mlp=MLPClassifier(glove.vectors,100)
mlp.load_state_dict(checkpoint)
checkpoint = torch.load('cnn.pt')
cnn=CNNClassifier(glove.vectors,100,3,230,4,230)
cnn.load_state_dict(checkpoint)

def greet(Sentence):
    tokens = Sentence.split()
    token_ints = [glove.stoi.get(tok, len(glove.stoi)-1) for tok in tokens]
    token_tensor = torch.LongTensor(token_ints).view(-1,1)
    print(token_tensor)
    pred=mlp(token_tensor)
    probmlp=round(pred.item()*10000)/100
    pred=cnn(token_tensor)
    probcnn=round(pred.item()*10000)/100
    
    return "This sentence is predicted to be "+str(probmlp)+"% subjective by the MLP Classifier!\nThis sentence is predicted to be "+str(probcnn)+"% subjective by the CNN Classifier!"
    
demo = gr.Interface(fn=greet, inputs="text", outputs="text")
demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
